In [ ]:
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from itertools import repeat
import networkx as nx
from gensim.models import Word2Vec
from tqdm import tqdm
from collections import Counter

In [ ]:
# all_data = pd.read_csv('./tcc_ceds_music.csv') #the dataset that you want to obtain a graph
all_data = pd.read_csv("https://gist.githubusercontent.com/ivangermanov/f882d48647cc7457d2cb99e0f27eb8b7/raw/9909e23b85bb146e78da1d7c4b71ea20b2262d55/processed.csv")

In [ ]:
all_data = all_data.sample(frac=1).reset_index(drop=True)
all_data

,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,night/time,shake the audience,family/gospel,romantic,communication,obscene,music,movement/places,light/visual perceptions,family/spiritual,like/girls,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age,lang,processed_lyrics
0,57416,al jarreau,thinkin' about it too,1978,jazz,land lovers dream poets dwell sail tomorrow ro...,33,0.002193,0.002193,0.292657,0.002193,0.002193,0.002193,0.002193,0.136552,0.002193,0.002193,0.002193,0.116430,0.036578,0.002193,0.387080,0.002193,0.672912,0.570110,0.119477,0.000508,0.942292,0.723715,sadness,0.600000,en,afford begin borrow conquer doubt doubter drea...
1,45620,wipers,voices in the rain,1982,blues,darkness fall leave aimlessly wander wonder tr...,20,0.003096,0.239152,0.003096,0.003096,0.003096,0.003096,0.003096,0.003096,0.003096,0.003096,0.003096,0.222800,0.003096,0.003096,0.398287,0.003096,0.420557,0.793375,0.031124,0.930162,0.826876,0.891889,sadness,0.542857,en,aimlessli begin dark fall follow leav rain str...
2,62616,car bomb,from the dust of this planet,2016,jazz,dark smile say warpath straight lonely side fe...,47,0.001548,0.488903,0.001548,0.001548,0.001548,0.001548,0.001548,0.001548,0.001548,0.295285,0.001548,0.001548,0.001548,0.001548,0.191044,0.001548,0.237518,0.791170,0.000414,0.019332,0.213726,0.899897,violence,0.057143,en,ash break close cold dark dust ear earth feel ...
3,28598,holly dunn,there goes my heart again,1986,country,go heart come hurt think learn look heart star...,65,0.057912,0.000863,0.000863,0.000863,0.000863,0.017474,0.000863,0.071445,0.029787,0.059268,0.000863,0.000863,0.000863,0.056889,0.543325,0.097760,0.646919,0.672461,0.048393,0.000001,0.912407,0.591579,sadness,0.485714,en,beat blue break come control doin fine forget ...
4,66075,311,inner light spectrum,1997,reggae,inside skull bother senselessness wish messiah...,71,0.001074,0.001074,0.441139,0.179943,0.043596,0.001074,0.031922,0.088661,0.001074,0.001074,0.001074,0.001074,0.001074,0.001074,0.098760,0.001074,0.720568,0.655898,0.201806,0.000146,0.557914,0.494479,world/life,0.328571,en,babylon bother choke chugalug come conscious c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17772,24140,dolly parton,touch your woman,1972,country,right disagree right speak mind anger want ins...,80,0.000711,0.000711,0.033352,0.073019,0.064950,0.037965,0.466176,0.154387,0.000711,0.019866,0.000711,0.000711,0.012883,0.084664,0.000711,0.046336,0.509369,0.607825,0.848393,0.000005,0.454864,0.195170,romantic,0.685714,en,alright anger arm aw belong blow bring busi di...
17773,37199,jason isbell,different days,2013,country,star picture runaways wall like couldn away aw...,100,0.015832,0.057227,0.424026,0.055066,0.000585,0.049514,0.031781,0.118509,0.000585,0.000585,0.129621,0.078600,0.000585,0.022977,0.000585,0.000585,0.494206,0.517422,0.553212,0.000082,0.178689,0.317296,world/life,0.100000,en,answer away babi believ benzodiazepin catch co...
17774,45577,wipers,can this be,1981,blues,makin deal feel lookin confuse long long long ...,27,0.002288,0.002288,0.091437,0.705917,0.002288,0.002288,0.002288,0.002288,0.136009,0.002288,0.002288,0.002288,0.002288,0.002288,0.002288,0.032313,0.336077,0.846807,0.007750,0.737854,0.895919,0.926925,night/time,0.557143,en,confus deal feel life long lookin makin trail
17775,41591,ella fitzgerald,can't buy me love,1964,blues,ring friend make feel alright make feel alrigh...,48,0.001096,0.001096,0.001096,0.001096,0.113977,0.001096,0.001096,0.217958,0.445084,0.001096,0.001096,0.001096,0.001096,0.145714,0.001096,0.001096,0.398895,0.763685,0.836345,0.000000,0.808326,0.786780,obscene,0.800000,en,alright aw care feel friend kind make money ne...


In [ ]:
holder = all_data['release_date']
Counter(all_data['genre'])

Counter({'blues': 3014,
         'country': 3251,
         'hip hop': 611,
         'jazz': 2335,
         'pop': 4536,
         'reggae': 1614,
         'rock': 2416})

In [ ]:
#Obtain Node Dataframe Method
def obtainNodes(musicData): 
    wordNodes = obtainWordNodes(musicData)
    artistNodes = obtainArtistNodes(musicData)
    attributeNodes = obtainAttributeNodes(musicData)

    song_names = musicData['track_name'].tolist()
    genres = musicData['genre'].tolist()
    no_of_songs = len(musicData['artist_name'])
    no_of_words = len(wordNodes)
    no_of_artist = len(artistNodes)
    no_of_attribute = len(attributeNodes)
    type_list=[]
    type_list.extend(repeat('song',no_of_songs))
    type_list.extend(repeat('word',no_of_words))
    type_list.extend(repeat('artist',no_of_artist))
    type_list.extend(repeat('attribute',no_of_attribute))
    wordGenres = wordNodes['genre'].tolist()
    artistGenres = artistNodes['genre'].tolist()
    attributeGenres = attributeNodes['genre'].tolist()
    genres += wordGenres
    genres += artistGenres
    genres += attributeGenres
    node_ids = [*range(0, no_of_words + no_of_songs + no_of_artist + no_of_attribute)]
    node_names = [y for x in [song_names, wordNodes['word'].tolist(), artistNodes['word'].tolist(), attributeNodes['word'].tolist()] for y in x]
    list_of_tuples = list(zip(node_ids, node_names, type_list, genres))
    df = pd.DataFrame(list_of_tuples,
                  columns = ['node_id', 'node_name', 'node_type', 'genre'])
    return musicData, df, wordNodes
    
def obtainWordNodes(processedMusicData):
    unique_words = {}
    for i in range(len(processedMusicData['processed_lyrics'])):
        words = processedMusicData['processed_lyrics'][i].split(' ')
        for word in words:
            unique_words[word] = ''
    keyList = list(unique_words.keys())    
    valueList=list(unique_words.values())
    node_ids = [*range(0, len(unique_words))]
    list_of_tuples = list(zip(node_ids, keyList, valueList))
    df = pd.DataFrame(list_of_tuples,
                  columns = ['node_id', 'word', 'genre'])
    return df
def obtainArtistNodes(processedMusicData):
    unique_words = {}
    for i in range(len(np.unique(processedMusicData['artist_name']))):
        unique_words[processedMusicData['artist_name'][i]] = ''  
    keyList = list(unique_words.keys())
    
    valueList=list(unique_words.values())
    node_ids = [*range(0, len(unique_words))]
    list_of_tuples = list(zip(node_ids, keyList, valueList))
    df = pd.DataFrame(list_of_tuples,
                  columns = ['node_id', 'word', 'genre'])
    return df
def obtainAttributeNodes(processedMusicData):
    unique_words = {}  
    unique_words['artist'] = ''
    unique_words['lyric'] = ''
    keyList = list(unique_words.keys())
    
    valueList=list(unique_words.values())
    node_ids = [*range(0, len(unique_words))]
    list_of_tuples = list(zip(node_ids, keyList, valueList))
    df = pd.DataFrame(list_of_tuples,
                  columns = ['node_id', 'word' , 'genre'])
    return df

In [ ]:
processed_music, nodes, words = obtainNodes(all_data)
empty_list = [0.0 for i in range(0,100)]
vec = np.array(empty_list)

['jazz', 'pop', 'country', 'rock', 'blues', 'pop', 'country', 'rock', 'pop', 'blues', 'country', 'blues', 'reggae', 'rock', 'country', 'blues', 'pop', 'jazz', 'jazz', 'country', 'rock', 'rock', 'pop', 'pop', 'rock', 'country', 'pop', 'blues', 'reggae', 'blues', 'reggae', 'reggae', 'reggae', 'rock', 'rock', 'country', 'hip hop', 'rock', 'pop', 'rock', 'country', 'blues', 'pop', 'pop', 'reggae', 'blues', 'blues', 'pop', 'rock', 'country', 'country', 'reggae', 'jazz', 'rock', 'rock', 'jazz', 'pop', 'blues', 'rock', 'country', 'pop', 'country', 'rock', 'blues', 'blues', 'pop', 'country', 'pop', 'blues', 'pop', 'pop', 'jazz', 'pop', 'blues', 'pop', 'pop', 'pop', 'country', 'pop', 'pop', 'rock', 'reggae', 'pop', 'pop', 'rock', 'jazz', 'jazz', 'jazz', 'reggae', 'country', 'pop', 'blues', 'country', 'reggae', 'pop', 'blues', 'country', 'country', 'jazz', 'rock', 'reggae', 'rock', 'country', 'blues', 'blues', 'country', 'country', 'jazz', 'country', 'country', 'pop', 'pop', 'country', 'jazz', '

In [ ]:
G = nx.Graph()
wordCounter = 0
for i in tqdm(range(len(nodes))):
    G.add_node(i, 
              node_id = str(nodes.iloc[i]['node_id']), 
              node_name = str(nodes.iloc[i]['node_name']), 
              node_type = str(nodes.iloc[i]['node_type']), 
              genre = str(nodes.iloc[i]['genre']))

100%|██████████| 53280/53280 [00:30<00:00, 1764.92it/s]

53280


In [ ]:
#Construct edges Method
def get_node_indexes(song, nodes):
    list_words = []
    words = song.split(' ')
    for word in words:    
        list_words.append(int(nodes['node_id'][nodes['node_name'] == word].iloc[-1]))
    return list_words
def get_artist_index(song_artist, nodes):
    return nodes['node_id'][nodes['node_name'] == song_artist]
    
artist_node_index = len(nodes) - 2
lyrics_node_index = len(nodes) - 1

for i in tqdm(range(len(processed_music))):
    node_indexes = get_node_indexes(processed_music['processed_lyrics'][i],nodes)
    for j in node_indexes:
        G.add_edge(i, j)
        G.add_edge(j, lyrics_node_index)
    artist_index = get_artist_index(processed_music['artist_name'][i],nodes)
    for j in artist_index:
        G.add_edge(i, j)
        G.add_edge(j, artist_node_index)

100%|██████████| 17777/17777 [51:02<00:00,  5.80it/s]

794103


In [ ]:
nx.write_gml(G, "graph_node2vec.gml")